In [1]:
!pip install --upgrade torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113
!python -c "import torch; print(torch.__version__); print(torch.version.cuda);"
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.11.0+cu113.html

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 714.8 kB/s eta 0:00:000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 25.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 29.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0
    Uninstalling torchvision-0.12.0:
      Successfully uninstalled torchvision-0.12.0
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.11.0
    Uninstalling torchaudio-0.11.0:
      Successfully uninstalled torchaudio-0.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the followin

## Step1: Data Preprocessing
Detailed explaination can be found in data_exploration.ipynb

In [4]:
import numpy as np
import gc
import torch
import pyarrow as pa
from tqdm import tqdm
from pyarrow.parquet import ParquetFile
from sklearn.neighbors import kneighbors_graph
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [5]:
import glob
import pandas as pd
ds_dir = glob.glob('../input/boosted-top-tau-4/*')
X_jets = []
lables = []

for file_name in ds_dir:
    df = pd.read_parquet(file_name)
    X_jets.append(np.array(df['X_jets'].tolist()).astype(np.float32))
    lables.append(df['y'].to_numpy())
    del df

In [6]:
X_jets = np.array(X_jets).reshape((-1,125000))
lables = np.array(lables).reshape((-1,1))
X_data = X_jets.reshape((-1,125*125,8))

In [8]:
# uncommet this line below if choose to use full channels for node features
X_data = X_data[:,:,:5]
non_black_pixels_mask = np.any(X_data != 0., axis=-1)

node_list = []
for i, x in enumerate(X_data):
    node_list.append(x[non_black_pixels_mask[i]])

# clear cache after use due to RAM limits
del X_jets, X_data, non_black_pixels_mask

In [9]:
dataset = []
for i,nodes in enumerate(tqdm(node_list)):
    edges = kneighbors_graph(nodes, 15, mode='connectivity', include_self=True)
    c = edges.tocoo()
    edge_list = torch.from_numpy(np.vstack((c.row, c.col))).type(torch.long)
    edge_weight = torch.from_numpy(c.data.reshape(-1,1))
    y = lables[i]
    dataset.append(Data(x=torch.from_numpy(nodes), edge_index=edge_list, edge_attr=edge_weight, y=torch.from_numpy(y).long()))

100%|██████████| 9600/9600 [01:21<00:00, 118.09it/s]


In [10]:
del lables, node_list, edge_list, edge_weight, y
gc.collect()

551

In [12]:
rand_seed = 42
X_train, X_test = train_test_split(dataset, test_size=0.1, random_state = rand_seed)
X_train, X_val = train_test_split(X_train, test_size=0.1, random_state = rand_seed)
print(len(X_train), len(X_val), len(X_val))

7776 864 864


In [13]:
train_loader = DataLoader(X_train, batch_size=32, shuffle=True)
val_loader = DataLoader(X_val, batch_size=32, shuffle=False)
test_loader = DataLoader(X_test, batch_size=32, shuffle=False)
batch = next(iter(test_loader))
print("Batch:", batch)
print("Labels:", batch.y[:10])
print("Batch indices:", batch.batch[:40])

Batch: DataBatch(x=[43043, 5], edge_index=[2, 645645], edge_attr=[645645, 1], y=[32], batch=[43043], ptr=[33])
Labels: tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 1])
Batch indices: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


## Step2: Define Graph SAGE Model

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

In [15]:
from torch.nn import Linear
from torch_geometric.nn import SAGEConv, TopKPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
num_node_features = 5
num_classes = 2
class GCN(torch.nn.Module):
    def __init__(self, c_in, c_hidden, c_out, dp_rate_linear=0.3):
        super().__init__()
        torch.manual_seed(123)
        
        self.conv1 = SAGEConv(c_in, c_hidden)
        self.pool1 = TopKPooling(c_hidden, ratio=0.8)
        self.conv2 = SAGEConv(c_hidden, c_hidden)
        self.pool2 = TopKPooling(c_hidden, ratio=0.8)
        self.conv3 = SAGEConv(c_hidden, c_hidden)
        self.pool3 = TopKPooling(c_hidden, ratio=0.8)
        self.lin1 = torch.nn.Linear(c_hidden*2, c_hidden)
        self.lin2 = torch.nn.Linear(c_hidden, c_out)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()
        self.dp_rate_linear = dp_rate_linear
  
    def forward(self, x, edge_index, batch):       
        x = F.relu(self.conv1(x, edge_index))
        #x, edge_index, edge_attr, batch, perm, score[perm]

        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=self.dp_rate_linear, training=self.training)

        return x 

## Step3: Use PyTorch Lightning to Define Training Process

In [16]:
from torchmetrics.functional import auroc
class GraphLevelGNN(pl.LightningModule):
    
    def __init__(self, **model_kwargs):
        super().__init__()
        # Saving hyperparameters
        self.save_hyperparameters()
        
        self.model = GCN(**model_kwargs)
        self.loss_module = nn.BCEWithLogitsLoss() if self.hparams.c_out == 1 else nn.CrossEntropyLoss()
        self.auroc = auroc
        
    def forward(self, data, mode="train"):
        x, edge_index, batch_idx = data.x, data.edge_index, data.batch
        # print(data.x.shape, data.edge_index.shape, data.batch.shape)
        
        x = self.model(x, edge_index, batch_idx)
        x = x.squeeze(dim=-1)
        
        if self.hparams.c_out == 1:
            preds = (x > 0).float()
            data.y = data.y.float()
        else:
            preds = x.argmax(dim=-1)
        loss = self.loss_module(x, data.y)
        acc = (preds == data.y).sum().float() / preds.shape[0]
        auc = self.auroc(x, data.y, num_classes=2)
        return loss, acc, auc

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=3e-4, weight_decay=0) # High lr because of small dataset and small model
        return optimizer

    def training_step(self, batch, batch_idx):
        loss, acc, auc = self.forward(batch, mode="train")
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        self.log('train_auc', auc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc, auc = self.forward(batch, mode="val")
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        self.log('val_auc', auc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        loss, acc, auc = self.forward(batch, mode="test")
        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', acc, prog_bar=True)
        self.log('test_auc', auc, prog_bar=True)

In [17]:
CHECKPOINT_PATH = "./"
def train_graph_classifier(model_name, **model_kwargs):
    pl.seed_everything(42)
    
    # Create a PyTorch Lightning trainer with the generation callback
    root_dir = os.path.join(CHECKPOINT_PATH, "GraphLevel" + model_name)
    os.makedirs(root_dir, exist_ok=True)
    trainer = pl.Trainer(default_root_dir=root_dir,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc")],
                         gpus=1 if str(device).startswith("cuda") else 0,
                         max_epochs=20,
                         progress_bar_refresh_rate=5)

    # Check whether pretrained model exists. If yes, load it and skip training
    model = GraphLevelGNN(**model_kwargs)
    print(model)
    trainer.fit(model, train_loader, val_loader)
    model = GraphLevelGNN.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
    
    # Test best model on validation and test set
    # train_result = trainer.test(model, dataloaders=train_loader, verbose=False)
    val_result = trainer.test(model, dataloaders=val_loader, verbose=False)
    test_result = trainer.test(model, dataloaders=test_loader, verbose=False)
    result = {"test": test_result[0]['test_acc'], "valid": val_result[0]['test_acc']} 
    print(val_result)
    print(test_result)
    return model, result

In [18]:
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, result = train_graph_classifier(model_name="GCN", c_in=5, c_hidden=64, c_out=2)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=5)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"


GraphLevelGNN(
  (model): GCN(
    (conv1): SAGEConv(5, 64, aggr=mean)
    (pool1): TopKPooling(64, ratio=0.8, multiplier=1.0)
    (conv2): SAGEConv(64, 64, aggr=mean)
    (pool2): TopKPooling(64, ratio=0.8, multiplier=1.0)
    (conv3): SAGEConv(64, 64, aggr=mean)
    (pool3): TopKPooling(64, ratio=0.8, multiplier=1.0)
    (lin1): Linear(in_features=128, out_features=64, bias=True)
    (lin2): Linear(in_features=64, out_features=2, bias=True)
    (act1): ReLU()
    (act2): ReLU()
  )
  (loss_module): CrossEntropyLoss()
)


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/data.py:73: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 37375. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/data.py:73: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 37001. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"


Training: 0it [00:00, ?it/s]

...

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/data.py:73: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 43043. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/data.py:73: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 38576. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/data.py:73: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 41724. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from 

[{'test_loss': 0.41307148337364197, 'test_acc': 0.8281190991401672, 'test_auc': 0.8758936524391174}]
[{'test_loss': 0.416495144367218, 'test_acc': 0.814951479434967, 'test_auc': 0.8623855113983154}]


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/data.py:73: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 34894. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/data.py:73: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 37376. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
